In [1]:
#import

import pandas as pd

In [ ]:

# read clipboard
#option 1 : get the whole thing in clipboard, easier but doesn't work in remote env
df = pd.read_clipboard(sep='\t')
#option 2 : get the whole thing from excel

#took option 1

In [4]:
#concat PHYID

data_concat = pd.DataFrame(df['ID'] + "-" + df['Branch (ID)']).rename(columns={0: 'PHYID'}, inplace=False)
data_concat = data_concat.PHYID.str.replace(",","")

In [5]:
#then concat 2 frames

frames = [df, data_concat]
new_frame = pd.concat(frames, axis=1) #level=PHYID

In [6]:
# create a new grouped frame, sum all items and set new index before dumping JSON

required = ['PHYID','Product (Code)','Product (Name)','Serial','Number']
new_frame = pd.DataFrame(new_frame[required])
grouped = new_frame.groupby(['PHYID', 'Product (Code)', 'Product (Name)', 'Serial'], as_index=False, dropna=False).agg({'Number' : 'sum'}).fillna("NULL").set_index(keys=['PHYID','Product (Code)'])


In [77]:
#Create nested dict using defaultdict and loop
#thanks to : https://stackoverflow.com/questions/46205399/how-to-generate-n-level-hierarchical-json-from-pandas-dataframe

from collections import defaultdict
import json
nested_dict = lambda: defaultdict(dict)
nested = nested_dict()

for row in grouped.itertuples():
    nested[row.Index[0]][row.Index[1]] = {'Product (Code)' : row.Index[1],'Product (Name)' : row._1, 'Serial' : row.Serial, 'Number' : row.Number}

with open("stockout_data.json","w+") as file:
    json.dump(nested, file, indent=3)

#with open("readable_dup_allow.xlsx", 'wb') as file:
    #grouped.to_excel(file, sheet_name="1", index=True)



<H2>LEGACY CODE</H2>

In [ ]:
#my_df = df #df or df_excel

data_concat = pd.DataFrame(df['Stock Out (ID)'] + "-" + df['Branch (ID)']).rename(columns={0 : 'PHYID'}, inplace=False)
data_concat = data_concat['PHYID'].str.replace(",","")

In [ ]:
#read clipboard



data_cat = pd.concat([df,data_concat], axis=1)
result = data_cat.set_index('PHYID').rename(columns={'Good/Bad' : 'Good | Bad'}, inplace=False)
##need Booking ID, Stockout-ID, Branch ID,  Branch To Name, Comment
required = ['Booking ID','Stock Out (ID)','Branch (ID)','Branch To (Name)','Comment']
result['Booking ID'] = result['Booking ID'].str.replace('Booking-DHL ID : ','')
result = pd.DataFrame(result[required])

In [29]:
# 0 = ID33_Headoffice
# 1 = ID33_insure
# 2 = ID49
# 3 = ID49_Tradein

filedict = {0 : 'ID33_Headoffice', 1 : 'ID33_insure', 2 : 'ID49', 3 : 'ID49_Tradein'}


ID33_Headoffice.json


In [45]:
get_filedict = 3

with open(f'{filedict.get(get_filedict)}.json','w+', encoding='utf-8',) as file:
    result.to_json(file,orient='index',index=True, force_ascii=False, indent=2)

In [ ]:
import firebase_admin as fb
from firebase_admin import firestore
import pandas as pd

cred = fb.credentials.Certificate(rf"C:\ITEC_Support\my-work-python-script\Database_in_pandas\serviceAccount.json")
default_app = fb.initialize_app(cred)
db = firestore.client()



In [18]:
result = result.to_dict(orient='index')

In [ ]:
my_doc_Ref = db.collection("Data").document("ID:33_Insure")
my_doc_Ref.set(result)